# Libraries

In [4]:
import datetime as dt

import pandas as pd

from openbb_terminal.sdk import openbb
import quantstats as qs
import backtrader as bt

#  Functions

In [5]:
def openbb_data_to_bt_data(symbol, start_date, end_date):
    
    df = openbb.stocks.load(symbol, start_date=start_date, end_date=end_date)
    
    fn = f"{symbol.lower()}.csv"
    df.to_csv(fn)
    
    return bt.feeds.YahooFinanceCSVData(
        dataname=fn,
        fromdate=dt.datetime.strptime(start_date, '%Y-%m-%d'),
        todate=dt.datetime.strptime(end_date, '%Y-%m-%d')
    )


def last_day_of_month(any_day):
    # The day 28 exists in every month. 4 days later, it's always next month
    next_month = any_day.replace(day=28) + dt.timedelta(days=4)
    
    # subtracting the number of the current day brings us back one month
    return (next_month - dt.timedelta(days=next_month.day)).day

# Backtrader Strategy

In [6]:
class MonthlyFlows(bt.Strategy):
    
    params = (
        ("end_of_month", 23),
        ("start_of_month", 7),
    )
    
    def __init__(self):
        self.order = None
        self.dataclose = self.datas[0].close
        
    def notify_order(self, order):
        # No more orders
        self.order = None    
    
    def next(self):
        
        # Get today's date, day of month, and last day of current month
        dt_ = self.datas[0].datetime.date(0)
        dom = dt_.day
        ldm = last_day_of_month(dt_)
        
        # If an order is pending, exit
        if self.order:
            return
        
        # Check if we are in the market
        if not self.position:
            
            # We're in the first week of the month, sell
            if dom <= self.params.start_of_month:

                # Sell the entire portfolio
                self.order = self.order_target_percent(target=-1)
                
                print(f"Created SELL of {self.order.size} at {self.data_close[0]} on day {dom}")
            
            # We're in the last week of the month, buy
            if dom >= self.params.end_of_month:

                # Buy the entire portfolio
                self.order = self.order_target_percent(target=1)
                
                print(f"Created BUY of {self.order.size} {self.data_close[0]} on day {dom}")
        
        # We are not in the market
        else:
            
            # If we're long
            if self.position.size > 0:
                
                # And not within the last week of the month, close
                if not self.params.end_of_month <= dom <= ldm:
                    
                    print(f"Created CLOSE of {self.position.size} at {self.data_close[0]} on day {dom}")

                    self.order = self.order_target_percent(target=0.0)
            
            # If we're short
            if self.position.size < 0:
                
                # And not within the first week of the month, close
                if not 1 <= dom <= self.params.start_of_month:
                    
                    print(f"Created CLOSE of {self.position.size} at {self.data_close[0]} on day {dom}")

                    # self.order = self.close()
                    self.order = self.order_target_percent(target=0.0)

# Backtest

In [7]:
data = openbb_data_to_bt_data(
    "TLT", 
    start_date="2002-01-01",
    end_date="2022-06-30"
)

cerebro = bt.Cerebro(stdstats=False)

cerebro.adddata(data)
cerebro.broker.setcash(1000.0)

cerebro.addstrategy(MonthlyFlows)

cerebro.addobserver(bt.observers.Value)

cerebro.addanalyzer(
    bt.analyzers.Returns, _name="returns"
)
cerebro.addanalyzer(
    bt.analyzers.TimeReturn, _name="time_return"
)

backtest_result = cerebro.run()

Loading Daily data for TLT with starting period 2002-07-30.

Created BUY of 24 41.24 on day 30
Created CLOSE of 24 at 41.99 on day 1
Created SELL of -23 at 42.42 on day 2
Created CLOSE of -23 at 42.34 on day 8
Created BUY of 23 43.45 on day 23
Created CLOSE of 23 at 44.73 on day 3
Created SELL of -23 at 44.72 on day 4
Created CLOSE of -23 at 44.37 on day 9
Created BUY of 23 45.77 on day 23
Created CLOSE of 23 at 45.36 on day 1
Created SELL of -23 at 45.62 on day 2
Created CLOSE of -23 at 45.74 on day 8
Created BUY of 24 43.0 on day 23
Created CLOSE of 24 at 44.04 on day 1
Created SELL of -24 at 43.85 on day 4
Created CLOSE of -24 at 45.3 on day 8
Created BUY of 23 43.99 on day 25
Created CLOSE of 23 at 43.86 on day 2
Created SELL of -23 at 43.93 on day 3
Created CLOSE of -23 at 44.59 on day 9
Created BUY of 22 44.98 on day 23
Created CLOSE of 22 at 44.62 on day 2
Created SELL of -22 at 44.73 on day 3
Created CLOSE of -22 at 44.99 on day 8
Created BUY of 22 45.23 on day 23
Created CLOSE of 22 at 45.61 on day 3
Created SELL of -22 at 45.81 on day 

# Convert result to DF

In [8]:
# Get the strategy returns as a dictionary
returns_dict = backtest_result[0].analyzers.time_return.get_analysis()

# Convert the dictionary to a DataFrame
returns_df = (
    pd.DataFrame(
        list(returns_dict.items()),
        columns = ["date", "return"]
    )
    .set_index("date")
)

# Assess results

In [10]:
bench = openbb.stocks.load(
    "TLT",
    start_date="2002-01-01",
    end_date="2022-06-30"
)["Adj Close"]

qs.reports.metrics(
    returns_df,
    benchmark=bench,
    mode="full"
)

Loading Daily data for TLT with starting period 2002-07-30.

                           Strategy    Benchmark
-------------------------  ----------  -----------
Start Period               2002-07-30  2002-07-30
End Period                 2022-06-29  2022-06-29
Risk-Free Rate             0.0%        0.0%
Time in Market             55.0%       100.0%

Cumulative Return          114.8%      171.88%
CAGR﹪                     3.91%       5.15%

Sharpe                     0.47        0.43
Prob. Sharpe Ratio         98.25%      97.19%
Smart Sharpe               0.45        0.41
Sortino                    0.68        0.61
Smart Sortino              0.65        0.59
Sortino/√2                 0.48        0.43
Smart Sortino/√2           0.46        0.41
Omega                      1.12        1.12

Max Drawdown               -27.69%     -34.76%
Longest DD Days            2555        1086
Volatility (ann.)          8.97%       14.06%
R^2                        0.01        0.01
Information Ratio          -0.01       -0.01
Calmar                     0.14     